In [1]:
%run "../app/dreams"

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from datetime import datetime

In [68]:
from re import sub
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '', s)
    return s



In [5]:
tested_hts = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "tested_on_given_date")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

In [6]:
tested_hts_21 = tested_hts[tested_hts.fiscal_year != "Q3fy20-Q4fy20"]

## Nouvelles Filles HTS

In [7]:
hts_new = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "errata")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

In [8]:
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [9]:
hts_new.to_excel(f"verification_{date_du_jour}.xlsx",index=False,na_rep="NULL")

## Oldest

> verification

Pick the verification as the starting point

In [10]:
old_hts = pd.read_excel("verification.xlsx")

In [11]:
old_hts.code.count()
hts_new.code.count()
tested_hts.code.count()
tested_hts_21.code.count()

4799

5149

1063

301

In [12]:
hts_new.date_entevyou.min().strftime("%Y-%m-%d")
hts_new.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-08-21'

In [13]:
old_hts.date_entevyou.min().strftime("%Y-%m-%d")
old_hts.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

In [14]:
tested_hts.date_entevyou.min().strftime("%Y-%m-%d")
tested_hts.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-18'

'2021-07-26'

In [15]:
tested_hts_21.date_entevyou.min().strftime("%Y-%m-%d")
tested_hts_21.date_entevyou.max().strftime("%Y-%m-%d")

'2020-10-02'

'2021-07-26'

In [16]:
howmuch_htsNew_isin_htsold = hts_new[hts_new.code.isin(old_hts.code)]

In [17]:
howmuch_htsNew_isnot_htsold = hts_new[~hts_new.code.isin(old_hts.code)]

In [18]:
howmuch_tested_isin_htsold = tested_hts[tested_hts.code.isin(old_hts.code)]

In [19]:
howmuch_tested21_isin_htsold =  tested_hts_21[tested_hts_21.code.isin(old_hts.code)]

In [20]:
howmuch_htsNew_isin_htsold.code.count()
howmuch_htsNew_isnot_htsold.code.count()
howmuch_tested_isin_htsold.code.count()
howmuch_tested21_isin_htsold.code.count()

4342

807

379

179

In [21]:
howmuch_htsNew_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_htsNew_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

In [22]:
howmuch_htsNew_isnot_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_htsNew_isnot_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-05-28'

'2021-08-21'

In [23]:
howmuch_tested_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_tested_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-18'

'2021-05-18'

In [24]:
howmuch_tested21_isin_htsold.date_entevyou.min().strftime("%Y-%m-%d")
howmuch_tested21_isin_htsold.date_entevyou.max().strftime("%Y-%m-%d")

'2020-10-02'

'2021-05-18'

## Report Component

In [25]:
hts_new_communes = []
for columns in hts_new.actual_commune.unique():
    globals()[f"hts_new_{name_handler(columns)}"] = hts_new[hts_new.actual_commune == columns]
    hts_new_communes.append(globals().get(f"hts_new_{name_handler(columns)}"))

In [26]:
new_communes_general = hts_new.actual_commune.unique().tolist()
hts_new_dict = {}
for columns, communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    hts_new_dict[f"{name_handler(communes)}"] =  globals().get(f"hts_new_{name_handler(columns)}").code.count()
    
    
hts_new_df = pd.DataFrame.from_dict(hts_new_dict, orient="index", columns=None)
hts_new_df.columns = ["nombre_filles_pour_depistage_hts"]
hts_new_df

,nombre_filles_pour_depistage_hts
Dessalines,191
Kenscoff,491
Gressier,285
CapHaïtien,245
Milot,79
PlaineduNord,110
QuartierMorin,47
Limonade,124
Verrettes,581
Liancourt,43


In [27]:
liste_hts_new = pd.ExcelWriter(f"rapportage_liste_filles_hts_{date_du_jour}.xlsx",engine="xlsxwriter")
hts_new_df.to_excel(liste_hts_new,sheet_name="Total_filles_pr_commune",index=True)
hts_new.to_excel(liste_hts_new,sheet_name="liste_des_filles_global",index=False,na_rep="NULL")
for columns,communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    globals().get(f"hts_new_{name_handler(columns)}").to_excel(liste_hts_new,sheet_name=f"{name_handler(communes)}",index=False,na_rep="NULL")
liste_hts_new.save()
#liste_hts_new.close()

# Filles restant a depister par Hub

- old data verification an
- depistage effectivement retracable sur le systeme
- > le reste des depistage a faire par hub maintenant

In [58]:
old_hts.code.count()
terrain_test = pd.read_excel('../the_services/depistage_vih_1824.xlsx')
# 18-24 tester so far
terrain_test.code.count()
personne_4799_tester = old_hts[old_hts.code.isin(terrain_test.code)]
personne_4799_tester.code.count()
personne_4799_non_tester = old_hts[~old_hts.code.isin(terrain_test.code)]
personne_4799_non_tester.code.count()


4799

1063

379

4420

In [69]:
personne_4799_non_tester_hub = []
for columns in personne_4799_non_tester.actual_hub.unique():
    globals()[f"personne_4799_non_tester_{name_handler(columns)}"] = personne_4799_non_tester[personne_4799_non_tester.actual_hub == columns]
    personne_4799_non_tester_hub.append(globals().get(f"personne_4799_non_tester_{name_handler(columns)}"))

In [70]:
new_hub_general = personne_4799_non_tester.actual_hub.unique().tolist()
personne_4799_non_tester_dict = {}
for columns, hubs in zip(personne_4799_non_tester.actual_hub.unique(), new_hub_general):
    personne_4799_non_tester_dict[f"{name_handler(hubs)}"] =  globals().get(f"personne_4799_non_tester_{name_handler(columns)}").code.count()
    
    
personne_4799_non_tester_df = pd.DataFrame.from_dict(personne_4799_non_tester_dict, orient="index", columns=None)
personne_4799_non_tester_df.columns = ["nombre_filles_pour_depistage_hts"]
personne_4799_non_tester_df

,nombre_filles_pour_depistage_hts
CollègeLeopoldSegharSengor,80
LocalSOHADERK,14
EgliseCommunautairecatholiquedeRobin,109
InstitutionMixteWigSaintPierre,59
InstitutionMixteWigSaintPierreMariani3,10
...,...
ecoletheligenesteve,6
Ecoletheligenesteve,5
CollegehenryChristoph,12
TEKNIK,5


In [163]:
def excel_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single space
    s = sub(r"\s+", '', s)
    #s = sub(r"^(\b[A-Z]\w*\s*)+$",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)
    s = sub(r"\b(\w)",'',s)

    return s

In [150]:
import xlsxwriter

In [164]:

liste_personne_4799_non_tester = pd.ExcelWriter(f"rapportage_liste_filles_hts_restant_{date_du_jour}.xlsx",engine="xlsxwriter")
personne_4799_non_tester_df.to_excel(liste_personne_4799_non_tester,sheet_name="Total_filles_pr_commune",index=True)
personne_4799_non_tester.to_excel(liste_personne_4799_non_tester,sheet_name="liste_des_filles_global",index=False,na_rep="NULL")
for columns,hub in zip(personne_4799_non_tester.actual_hub.unique(), new_hub_general):
    try:
        globals().get(f"personne_4799_non_tester_{name_handler(columns)}").to_excel(liste_personne_4799_non_tester,sheet_name=f"{name_handler(hub)}",index=False,na_rep="NULL")
    except xlsxwriter.exceptions.InvalidWorksheetName :
        globals().get(f"personne_4799_non_tester_{name_handler(columns)}").to_excel(liste_personne_4799_non_tester,sheet_name=f"{excel_handler(hub)}",index=False,na_rep="NULL")
    except xlsxwriter.exceptions.DuplicateWorksheetName :
        globals().get(f"personne_4799_non_tester_{name_handler(columns)}").to_excel(liste_personne_4799_non_tester,sheet_name=f"{excel_handler(hub)}_1",index=False,na_rep="NULL")
                
liste_personne_4799_non_tester.save()